In [2]:
import subprocess
from pathlib import Path

import geopandas
import shapely

In [3]:
etl_path = Path('.').absolute().parent.parent
raw_dir = etl_path / 'raw_data'
boundary_dir = etl_path / 'raw_data' / 'boundaries'

In [3]:
layers = [
    # ('admin-level-0', ['GID_0', 'NAME_0', 'geometry']),
    # ('admin-level-1', ['GID_0', 'NAME_0', 'GID_1', 'NAME_1', 'geometry']),
    # ('admin-level-2', ['GID_0', 'NAME_0', 'GID_1', 'NAME_1', 'GID_2', 'NAME_2', 'geometry']),
    ('hybas_lev12_v1c_with_gadm_codes', ['HYBAS_ID', 'GID_0', 'NAME_0', 'GID_1', 'NAME_1',  'GID_2', 'NAME_2', 'geometry']),
]

for layer, cols in layers:
    print("Processing", layer)
    df = geopandas.read_parquet(boundary_dir / f"{layer}.geoparquet")
    if layer == 'hybas_lev12_v1c_with_gadm_codes':
        df.reset_index(inplace=True)
    df = df[cols].copy()
    print("Read")
    bounds = df.bounds # minx  miny  maxx  maxy
    bbox = shapely.box(bounds.minx, bounds.miny, bounds.maxx, bounds.maxy)
    df['bbox_wkt'] = geopandas.GeoSeries(bbox).to_wkt()
    print("Bounds")

    # df.to_file(boundary_dir / f"{layer}.geojsonld", driver='GeoJSONSeq', engine="pyogrio")
    print("Wrote poly")

    df.geometry = df.geometry.centroid
    print("Centroids")
    df.to_file(boundary_dir / f"{layer}_points.geojsonld", driver='GeoJSONSeq', engine="pyogrio")
    print("Wrote points")


Processing hybas_lev12_v1c_with_gadm_codes
Read
Bounds
Wrote poly


/tmp/ipykernel_3079473/1301827610.py:23: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df.geometry = df.geometry.centroid


Centroids
Wrote points


In [ ]:
layers = [
    ('admin-level-0', 'adm0'),
    ('admin-level-1', 'adm1'),
    ('admin-level-2', 'adm2'),
    ('hybas_lev12_v1c_with_gadm_codes', 'hybas'),
]

for geojson, mbtiles in layers:
    for suffix in ["", "_points"]:
        subprocess.run([
            "ln", "-s", f"{boundary_dir}/{geojson}.geojsonld", f"{boundary_dir}/{mbtiles}{suffix}.json"
        ])
        subprocess.run([
            "tippecanoe",
            "--read-parallel",
            "--force",
            "--drop-densest-as-needed",
            "-n",
            f"{mbtiles}{suffix}",
            "-zg",
            "-o",
            f"{etl_path}/../tileserver/vector/data/{mbtiles}{suffix}.mbtiles",
            f"{boundary_dir}/{mbtiles}{suffix}.json"
        ])

For layer 0, using name "adm0_points"
256 features, 9608 bytes of geometry and attributes, 35940 bytes of string pool, 0 bytes of vertices, 0 bytes of nodes
Choosing a maxzoom of -z0 for features typically 2622777 feet (799423 meters) apart, and at least 524812 feet (159963 meters) apart
  99.8%  0/0/0  
For layer 0, using name "adm1_points"
3610 features, 165508 bytes of geometry and attributes, 504377 bytes of string pool, 0 bytes of vertices, 0 bytes of nodes
Choosing a maxzoom of -z1 for features typically 357836 feet (109069 meters) apart, and at least 48955 feet (14922 meters) apart
  88.0%  1/1/0  
For layer 0, using name "adm2_points"
45962 features, 2716120 bytes of geometry and attributes, 6575061 bytes of string pool, 0 bytes of vertices, 0 bytes of nodes
Choosing a maxzoom of -z2 for features typically 94255 feet (28729 meters) apart, and at least 20900 feet (6371 meters) apart
tile 1/1/0 size is 536750 (probably really 536750) with detail 12, >500000    
Going to try keepi